# TP Decision Trees

Save the notebook as either PDF or HTML and make sure all the results are saved correctly (I won't run them and the original format does not save the results automatically), **and put your name in the filename**.

<div class="alert alert-success"> 
<b>Questions are in green boxes.</b>
The maximum time you should spend on each question is given as indication only. If you take more time than that, then you should come see me.
</div>
<div class="alert alert-info" role="alert"><b>Analyzes are in blue boxes.</b> You should comment on your results in theses boxes (Is it good? Is it expected? Why do we get such result? Why is it different from the previous one? etc)
</div>

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import gzip
import pickle
import matplotlib.pyplot as plt
import time

For this lab, we will use the [MNIST dataset (~15Mo)](https://educnet.enpc.fr/mod/resource/view.php?id=44975). It consists of 28x28 images (loaded as a 784 vector) and the associated label for training, validation and test sets. 

The following code loads all samples from the MNIST archive into several arrays, then displays the first sample from the training set as an image. We will limit ourselves to the first 1000 samples of train and 1000 samples of validation.

In [ ]:
# Load the dataset
data = np.load('mnist1k.npz')
X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_train_bin = data['X_train_bin']
y_train_bin = data['y_train_bin']
X_val_bin = data['X_val_bin']
y_val_bin = data['y_val_bin']
N_train = len(y_train)
N_val = len(y_val)
plt.imshow(X_train[0,:].reshape(28,28))
print(y_train[0])

## Implementing a randomized Decision Tree

<div class="alert alert-success"> 
    <b>Q1.</b> Implement the code of a function that finds the optimal threshold along a given dimension, using the $0-1$ loss with specified example weights and test it on the 150th dimension with the binary classification task. Compare it to assigning a unique label to all samples. <i>(Indicative time: 10 minutes for a slow version, but take the extra 20 minutes to have a parallel version testing all thresholds at once that runs instantaneously, it is worth it for the next questions.)</i>
</div>

In [ ]:
'''
takes arguements 
y_pred: prediction
y_true: true labels
weights: weights for each example
'''
@jax.jit
def zeroOneLoss(y_pred, y_true, weights):
    return (weights * (y_pred != y_true)).sum(axis=0)/(1e-12+weights.sum(axis=0))

In [ ]:
'''
takes arguments
X: training samples
y: training labels
dim: dimension to use
w: weight associated to each example (can be True/False or 1/0 to remove some examples)
returns the gain and the threshold
'''
@jax.jit
def findBestTh(X, y, dim, w):
    return

In [ ]:
%%time 


<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

We can vectorize over the dimension by using vmap. The batched function can now operate on a array of dimensions.

In [ ]:
batched_findBestTh = jax.vmap(findBestTh, (None, None, 0, None), 0)

<div class="alert alert-success"> 
    <b>Q2.</b> Implement the code of the Decision Tree class by using the previous functions. To achieve reasonable speed, loop only over dimensions that have variations (there is no threshold if all samples have the same value) in batches of 4 or 8. Do not split and slice the data but change the associated weights instead, it is faster (all functions have the same size of arrays and are thus compiled and optimized only once). Test it on the binary set to debug and then on the full set with a maximum depth of 4 and a minimum size of leaf of 100 to analyze and comment. <i>(Indicative time: It could take you 30 minutes to an hour to code and debug since it involves recursion. The model should train in less than 5 seconds on the binary set and under 5 minutes on the complete set)</i>
</div>

In [ ]:
class RandomizedDT():
    def __init__(self, percent_dimension=1.0, max_depth=8, max_size=20, verbose=False, space=0):
        self.percent_dimension = percent_dimension
        self.max_depth = max_depth
        if max_depth > 0:
            self.left = RandomizedDT(percent_dimension, max_depth=max_depth-1, max_size=max_size, verbose=verbose, space=space+2)
            self.right = RandomizedDT(percent_dimension, max_depth=max_depth-1, max_size=max_size, verbose=verbose, space=space+2)
        self.label = None
        self.max_size = max_size
        self.verbose = verbose
        self.space = " "*space
    
    '''
    train this decision tree on a random subset of dimensions (columns) of X with a maximum depth
    '''
    def fit(self, X, y, w=None):
        return       
    
    '''
    predict the set of samples
    '''
    def predict(self, X):
        if self.label is not None:
            return self.label * jnp.ones(len(X))
        return jnp.concatenate([self.left.predict([x]) if x[self.dim] < self.th else self.right.predict([x]) for x in X])

We first try on the training set reduced to digits 0 and 1 with all dimensions to check that our code works. 

In [ ]:
%%time


In [ ]:
%%time


<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

<div class="alert alert-success"> 
    <b>Q3.</b> Use cross-validation on the full digit dataset (0-9) to select a reasonnable depth between 2 and 8, using random splits of half the training set to save on training time. <i>(Indicative time: maximum 10 minutes to code, about 20 minutes to run)</i>

In [ ]:
%%time


<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## Random Forest

Next, we want to mitigate the tendancy of decision trees to overfit when the depth is too high and to underfit when the depth is too small by implementing random forests.

<div class="alert alert-success"> 
    <b>Q4.</b> Code a Random Forest of decision trees, each trained on a subset of the training set. Perform a corase cross-validation to set a reasonnable number of trees (5, 10, 15), percent of training data used (0.5, 0.75), percent of dimensions used (0.5, 0.75, 1.0) and depth 3. <i>(Indicative time: less than 10 minutes to code, takes more than 40 minutes to run)</i>
</div>

In [ ]:
class RandomForest():
    def __init__(self, nb_trees, percent_dataset=1., percent_dimension=1., max_depth=8):
        self.nb_trees = nb_trees
        self.percent_dataset = percent_dataset
        self.percent_dimension = percent_dimension
        self.max_depth = max_depth
        self.trees = []
        
    def fit(self, X, y):
        return
    
    def predict(self, X):
        y = []
        for dt in self.trees:
            y.append(dt.predict(X))
        y = jax.nn.one_hot(jnp.array(y), num_classes=10)
        return y.sum(axis=0).argmax(axis=1)

In [ ]:
%%time


<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## Boosting

To have a more efficient training procedure, we will remove the independance between the trees by using boosting

<div class="alert alert-warning"> 
    <b>Q5.</b> Code the BoostingClassifier that obtains a combination of Randomized Trees using AdaBoost. Each tree is trained using the weighted $0-1$ loss. To allow the tree combination, convert the output of each tree to a one-hot encoded vector. The output of the boosted trees is then the weighted sum of these one-hot vectors and the predicted class is the argmax. Test with the same parameters as the best Random Forest. <i>(Indicative time: about 30 minutes to code, runs about as fast as a random forest)</i>
</div>

In [ ]:
class BoostedTrees():
    def __init__(self, nb_trees, percent_dataset=1., percent_dimension=1., max_depth=8):
        self.nb_tress = nb_trees
        self.percent_dataset = percent_dataset
        self.percent_dimension = percent_dimension
        self.max_depth = max_depth
        
    def fit(self, X, y):
        return
    
    def predict(self, X):
        return

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

<div class="alert alert-warning"> 
    <b>Q6.</b> Perform a similar cross validation as for the Random Forest.
</div>

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## Visualization

In order to visualize the decision, we can produce an image the contains only the relevant information with respect to the decisions taken by a tree.

<div class="alert alert-success"> 
    <b>Q7.</b> For a trained tree, select a leaf and build an image that has a value of 1 for each pixel in the decision path that should be above the threshold, 0 for each pixels in the decision path that should be below the threshold and 0.5 everywhere else. For all classes, show an average all such images for each leaf corresponding to that class. <i>(Indicative time: about 20 minutes to code)</i>
</div>

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>